## Router
### Review
우리는 메시지를 상태(state)로 사용하고 도구(tool)가 연결된 채팅 모델을 활용하는 그래프를 구축했습니다.
<br>
우리는 이 그래프가 다음과 같은 기능을 수행할 수 있음을 확인했습니다:
- 도구 호출 반환
- 자연어 응답 반환

### Goals
우리는 이를 **라우터(router)**로 볼 수 있습니다. 즉, 채팅 모델이 사용자의 입력에 따라 직접 응답할지 또는 도구를 호출할지 결정하여 경로를 조정합니다.

이는 간단한 에이전트(agent)의 예시로, LLM이 도구를 호출하거나 직접 응답하는 방식으로 제어 흐름을 관리하는 역할을 수행합니다.

![Screenshot 2024-08-21 at 9.24.09 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac6543c3d4df239a4ed1_router1.png)


그래프 확장하기: 다양한 출력 방식 지원
이제 우리의 그래프를 두 가지 출력 방식 모두 처리할 수 있도록 확장해봅시다!

이를 위해 두 가지 아이디어를 활용할 수 있습니다:
1. 도구를 호출하는 노드 추가
2. 특정 조건에서 도구를 실행할 수 있도록 새로운 노드를 추가합니다. <br>
조건부(edge) 추가 <br>
채팅 모델의 출력을 확인하고, <br>
도구 호출이 필요한 경우 도구 호출 노드로 이동하고, <br>
도구 호출이 필요하지 않다면 바로 종료하도록 설정합니다.

In [12]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai, langchain_core, langgraph

In [ ]:
import os, getpass

os.environ['OPENAI_API_KEY'] = ""

In [18]:
%pip install langgraph


Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int     
    """
    return a * b

llm = ChatOpenAI(model = "gpt-4o")
llm_with_tools = llm.bind_tools([multiply])


In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode # 안찾아짐
from langgraph.prebuilt import tools_condition

# tools_condition 은 어떤 걸까?

# Node 
def tool_calling_llm(state: MessagesState): 
    return {"messages" : [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", ToolNode(tool_calling_llm))
builder.add_node("tools", ToolNode([multiply]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm", 
  # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
  # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END 
    tools_condition 
)
builder.add_edge("tools", END)
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [ ]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello, what is 2 multiplied by 2? .")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()